# Book lovers new app

# Project Description 
The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers.

You've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

**Connect to the database**

In [2]:
db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Study the tables**

In [3]:
books = pd.io.sql.read_sql('''SELECT * FROM books''', con = engine)
display(books.head())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
authors = pd.io.sql.read_sql('''SELECT * FROM authors''', con = engine)
display(authors.head())

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
publishers = pd.io.sql.read_sql('''SELECT * FROM publishers''', con = engine)
display(publishers.head())

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
ratings = pd.io.sql.read_sql('''SELECT * FROM ratings''', con = engine)
display(ratings.head())

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
reviews = pd.io.sql.read_sql('''SELECT * FROM reviews''', con = engine)
display(reviews.head())

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Find the number of books released after January 1, 2000.**

In [8]:
pd.io.sql.read_sql('''SELECT COUNT(book_id) FROM books WHERE publication_date > '2000-01-01' ''', con = engine)


,count
0,819


The number of books released after January 1st, 2000 is 819.

**Find the number of user reviews and the average rating for each book.**

In [9]:
 pd.io.sql.read_sql('''SELECT b.title,
 COUNT(r.review_id) AS reviews_num,
 AVG(ra.rating) AS avg_rating
 FROM books AS b
 INNER JOIN  reviews AS r on b.book_id = r.book_id
 INNER JOIN  ratings AS ra on b.book_id = ra.book_id 
 GROUP BY b.title
 ORDER BY reviews_num DESC''', con = engine)


,title,reviews_num,avg_rating
0,Twilight (Twilight #1),1120,3.662500
1,The Hobbit or There and Back Again,528,4.125000
2,The Catcher in the Rye,516,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,492,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,480,4.287500
...,...,...,...
988,The Adventures of Tom Sawyer and Adventures of...,2,5.000000
989,History of Beauty,2,2.500000
990,Lysistrata,2,4.000000
991,Executive Orders (Jack Ryan #8),2,3.500000


We have several book with average rating of 5!

**Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).**

In [10]:
 pd.io.sql.read_sql (''' SELECT 
                p.publisher,
                 COUNT(b.book_id) AS books_num
                FROM publishers AS p
                INNER JOIN books as b  ON p.publisher_id = b.publisher_id
                WHERE num_pages > 50
                GROUP BY publisher
                ORDER BY books_num DESC
                LIMIT 1
           ''', con = engine)

,publisher,books_num
0,Penguin Books,42


Penguin Books is the who released the greatest number of books with 42 books.

**Identify the author with the highest average book rating: look only at books with at least 50 ratings.**

In [11]:
 pd.io.sql.read_sql (''' 
 SELECT a.author,
 AVG(r.rating)
 FROM authors AS a 
 INNER JOIN books AS b ON  a.author_id = b.author_id
 INNER JOIN ratings AS r ON  b.book_id =  r.book_id
 GROUP BY a.author
 HAVING COUNT(r.rating) >= 50
 ORDER BY AVG(r.rating) DESC
 LIMIT 1
''', con = engine)

,author,avg
0,Diana Gabaldon,4.3


The author with the highest average book rating is Diana Gabaldon with 4.3 average rating.

**Find the average number of text reviews among users who rated more than 50 books.**

In [12]:
 pd.io.sql.read_sql (''' 
 SELECT
AVG(reviews_num) AS avg_reviews
FROM
(SELECT
username,
COUNT(DISTINCT review_id) AS reviews_num
FROM reviews
WHERE username IN 
(SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(DISTINCT book_id) > 50)
GROUP BY username) AS a
''', con = engine)

,avg_reviews
0,24.333333


The average number of text reviews among users who rated more than 50 books is 24.33.